In [75]:
#!pip install portalocker>=2.0.0

In [76]:
#pip install scikit-plot

In [77]:
import re

In [78]:
from torch.utils.data import Dataset, DataLoader

In [79]:
import gensim.downloader

In [80]:
import torch

In [81]:
w2v = gensim.downloader.load('word2vec-google-news-300')

In [82]:
torch.tensor(w2v["test"])

tensor([-1.4258e-01, -3.6865e-02,  1.3574e-01, -6.2012e-02,  7.9590e-02,
         1.9043e-02, -8.1543e-02, -1.2793e-01, -2.9541e-02,  2.3633e-01,
        -1.2158e-01, -2.1484e-01,  1.2988e-01, -2.7100e-02, -5.2002e-02,
         2.1582e-01, -1.8164e-01,  5.1025e-02, -1.6016e-01, -1.7676e-01,
         1.8311e-02, -4.1260e-02, -2.3242e-01, -1.0315e-02,  1.4551e-01,
         5.2490e-02, -3.9648e-01, -1.9287e-02,  2.5177e-03, -1.2695e-02,
        -4.3945e-02,  3.0762e-02,  9.5703e-02, -1.7578e-01,  1.0437e-02,
         1.8945e-01, -2.3633e-01,  4.3701e-02,  2.8125e-01, -2.0752e-02,
        -1.8164e-01, -2.1777e-01,  2.3340e-01,  5.2979e-02, -1.1377e-01,
         9.3994e-03, -1.4941e-01,  1.9922e-01, -1.7578e-01,  3.1641e-01,
         8.1055e-02, -6.1279e-02, -1.5234e-01, -1.8188e-02,  8.2520e-02,
         8.7402e-02, -1.1865e-01, -2.5977e-01, -1.6846e-02,  1.8799e-02,
         1.3611e-02, -2.3926e-01, -6.7871e-02, -8.1543e-02,  2.1875e-01,
         6.6406e-02,  1.2793e-01,  1.6406e-01,  2.2

In [83]:
from gensim.parsing.preprocessing import (preprocess_string,
                                          strip_tags,
                                          strip_punctuation,
                                          strip_multiple_whitespaces,
                                          strip_numeric,
                                          remove_stopwords)


# We pick a subset of the default filters,
# in particular, we do not take
# strip_short() and stem_text().
FILTERS = [strip_punctuation,
           strip_tags,
           strip_multiple_whitespaces,
           strip_numeric,
           remove_stopwords]

# See how the sentece is transformed into tokes (words)
preprocess_string('This is a "short" text!', FILTERS)

['This', 'short', 'text']

# Dataset Creation

In [84]:
import pandas as pd
#Load All_Beauty training, validation and testing split using read_csv from pandas library
#Write your code here
all_beauty_training_data = pd.read_csv(
    filepath_or_buffer='./data/All_Beauty_Split_train.csv'
)

all_beauty_test_data = pd.read_csv(
    filepath_or_buffer='data/All_Beauty_Split_test.csv'
)

all_beauty_validation_data = pd.read_csv(
    filepath_or_buffer='data/All_Beauty_Split_val.csv'
)

In [85]:
#Load Gift Card training, validation and testing split using read_csv from pandas library
#Write your code here
gift_cards_training_data = pd.read_csv(
    filepath_or_buffer='./data/Gift_Cards_Split_train.csv'
)

gift_cards_test_data = pd.read_csv(
    filepath_or_buffer='data/Gift_Cards_Split_test.csv'
)

gift_cards_validation_data = pd.read_csv(
    filepath_or_buffer='data/Gift_Cards_Split_val.csv'
)

In [86]:
#Concatenate the two splits to get a combined dataframe
#Write your code here
trainig_data = pd.concat([all_beauty_training_data, gift_cards_training_data], ignore_index=True)
test_data = pd.concat([all_beauty_test_data, gift_cards_test_data], ignore_index=True)
validation_data = pd.concat([all_beauty_validation_data, gift_cards_validation_data], ignore_index=True)

In [87]:
#Creating dataset class
class reviews(Dataset):
    def __init__(self,split):
        super(reviews,self).__init__()

        if split == 'train':
        #Give train dataframe
            df_data = trainig_data
        elif split == 'val':
        #Give validation dataframe
            df_data = validation_data
        elif split == 'test':
        #Give test dataframe
            df_data = test_data
        else:
            raise Exception(f"wrong split: {split}")
        #List for containing (input,output) pairs
        self.data = []
        #Make a list of ratings from df_data
        ratings = df_data['Rating']
        #Make a list of reviews from df_data
        text = df_data['Text']
        for i in range(len(text)):
            #Creating list of tokens for each review
            txt = preprocess_string(text[i])
            #Appending (input,output) pairs to list
            self.data.append([txt,ratings[i]])

    def vectorise(self,txt):
        #Final tensor
        X_tensor = torch.zeros(300)
        l=0
        for i in txt:
          #Check if the word is in the vocabulary of word2vec
          if i in w2v:
            l = l+1
            #If so add it to the X_tensor
            X_tensor += torch.tensor(w2v[i])
        #Divide X_tensor by number of vectors that were added. This is called mean pooling
        X_tensor = X_tensor / l
        return X_tensor

    def __getitem__(self,index):
        sent_tensor = self.vectorise(self.data[index][0])
        return [sent_tensor,(self.data[index][1]-1)]
    def __len__(self):
        return len(self.data)

In [88]:
import torch

target_classes = ["1", "2", "3", "4", "5"]

train_dataset = reviews("train")
val_dataset = reviews("val")
test_dataset = reviews("test")

train_loader = DataLoader(train_dataset, batch_size=128)
val_loader = DataLoader(val_dataset, batch_size=128)
test_loader  = DataLoader(test_dataset, batch_size=128)

In [89]:
for X, Y in train_loader:
    print(X.shape, Y.shape)
    break

torch.Size([128, 300]) torch.Size([128])


# Model Training

In [102]:
from torch import nn
from torch.nn import functional as F

#Embedding dimension of Word2vec
embed_len = 300

#Create a Neural Network for mapping the reviews to probabilities of different ratings
class EmbeddingClassifier(nn.Module):
    def __init__(self):
        super(EmbeddingClassifier, self).__init__()
        #Create a Neural Network with input dimension embed_len, hidden dimensions 256,128,64 and output dimension the length of target class.
        #Use Relu as the activation function between each layer.
        self.seq = nn.Sequential(
            nn.Linear(embed_len, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, len(target_classes)),
            nn.Softmax(1)
        )

    def forward(self, X_batch):
        return self.seq(X_batch)

In [100]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import gc

def CalcValLossAndAccuracy(model, loss_fn, val_loader):
    with torch.no_grad():
        Y_shuffled, Y_preds, losses = [],[],[]
        for X, Y in val_loader:
            #Predict output probabilities for all target classes for input X
            preds = model(X)
            loss = loss_fn(preds, Y.long())
            losses.append(loss.item())

            Y_shuffled.append(Y)
            #Get the predicted class from the output probabilities: The class with the highest probability. Append to Y_preds list(use argmax)


        Y_shuffled = torch.cat(Y_shuffled)
        Y_preds = torch.cat(Y_preds)

        print("Valid Loss : {:.3f}".format(torch.tensor(losses).mean()))
        print("Valid Acc  : {:.3f}".format(accuracy_score(Y_shuffled.detach().numpy(), Y_preds.detach().numpy())))

def TrainModel(model, loss_fn, optimizer, train_loader, val_loader, epochs=10):
    for i in range(1, epochs+1):
        losses = []
        for X, Y in tqdm(train_loader):
            #Predict output probabilities for all target classes for input X
            Y_preds = model(X)
            loss = loss_fn(Y_preds, Y.long())
            losses.append(loss.item())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if i%5==0:
            print("Train Loss : {:.3f}".format(torch.tensor(losses).mean()))
            CalcValLossAndAccuracy(model, loss_fn, val_loader)

In [106]:
from torch.optim import Adam

epochs = 25
learning_rate = 1e-3

loss_fn = nn.CrossEntropyLoss()
embed_classifier = EmbeddingClassifier()
optimizer = Adam(embed_classifier.parameters(), lr=learning_rate)

TrainModel(embed_classifier, loss_fn, optimizer, train_loader, test_loader, epochs)

  0%|          | 0/38 [00:00<?, ?it/s]

100%|██████████| 38/38 [00:01<00:00, 28.53it/s]


Train Loss : nan


RuntimeError: torch.cat(): expected a non-empty list of Tensors

# Evaluation

In [ ]:
def MakePredictions(model, loader):
    Y_shuffled, Y_preds = [], []
    for X, Y in loader:
        preds = model(X)
        Y_preds.append(preds)
        Y_shuffled.append(Y)
    gc.collect()
    Y_preds, Y_shuffled = torch.cat(Y_preds), torch.cat(Y_shuffled)

    return Y_shuffled.detach().numpy(), F.softmax(Y_preds, dim=-1).argmax(dim=-1).detach().numpy()

Y_actual_val, Y_preds_val = MakePredictions(embed_classifier, val_loader)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

#Write code to get full classification report and print a confusion matrix for validation set

In [ ]:
from sklearn.metrics import confusion_matrix
import scikitplot as skplt
import matplotlib.pyplot as plt
import numpy as np

#Plot a confusion matrix for the validation set

In [ ]:
Y_actual_test, Y_preds_test = MakePredictions(embed_classifier, test_loader)

In [ ]:
#Write code to get full classification report and print a confusion matrix for test set

In [ ]:
#Plot a confusion matrix for the test set